In [1]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain.llms import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from huggingface_hub import hf_hub_download
import textwrap
import glob
import os

In [2]:
os.environ['OPENAI_API_KEY'] = ''

In [3]:
model = OpenAI(model_name="gpt-4") 

c:\Users\roybm\miniconda3\lib\site-packages\langchain\llms\openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\roybm\miniconda3\lib\site-packages\langchain\llms\openai.py:696: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [4]:
import json

data = None
with open("../data/criminal_law.json", "r") as f:
    data = json.load(f)

In [5]:
type(data)

dict

In [6]:
[book for book in data]

['Book I', 'Book II']

In [7]:
texts = []

for book in data:
    for part in data[book]:
        for chapter in data[book][part]['chapters']:
            for section in data[book][part]['chapters'][chapter]['sections']:
                texts.append(
                    Document(
                        page_content=data[book][part]['chapters'][chapter]['sections'][section]['content'], 
                        page_title=data[book][part]['chapters'][chapter]['sections'][section]['title'], 
                        metadata={"book": book, "part": part, "chapter": chapter, "section": section, 'source': ', '.join([book, part, chapter, section])}
                    )
                )

print(len(texts))

393


In [8]:
hf_embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
chroma_index = Chroma.from_documents(texts, hf_embeddings)
print('Done')

Using embedded DuckDB without persistence: data will be transient


Done


In [13]:
results = chroma_index.similarity_search("What is the penalty for treason?", k=5)

for result in results:
    print(str(result.metadata['source']) + ":", result.page_content+"\n")

Book II, PART I, CHAPTER1, SECTION 103: Shall, if a citizen, be guilty of treason and, if a foreigner, be guilty of espionage, and shall in either case be punished with death whoever:
- a) instigates a foreign power to undertake hostilities against the Republic; or
- b) surrenders or offers to surrender to 3 foreign power or to its agents any troops. territory installations or equipment employer in the defence of the nation, or any dAlrightefence secret, or who in whatever manner acquires
such a secret with intent to surrender it to a foreign power,
- c) with intent to injure the defence of the nation, damages any
construction, installation or equipment, or commits any malpractice
liable to prevent their normal working or to lead to an accident.


Book II, PART I, CHAPTER IV, SECTION 153: 1. Whoever commits a contempt of the President of the Republic or any person exercising the whole or a part of their prerogatives, or of any foreign Head of State shall be punished with imprisonment f

In [16]:
chain = load_qa_with_sources_chain(model, chain_type='refine')
query = "While driving home drunk last night, I hit a woman crossing the road, what are the legal implications?"
documents = chroma_index.similarity_search(query)
result = chain({"input_documents": documents, "question": query})

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)).


In [21]:
result['output_text']

'The original answer seems to have covered most of the core points related to the specific unlawful act committed: driving under the influence and causing harm to another individual. The new context provided pertains to obstruction of public highways or waterways and refraining from repairing any damage caused, which doesn\'t directly impact the implications for the person who was driving while intoxicated and hit a pedestrian. Therefore, there is no need to update the answer as the additional context is not directly applicable in this particular scenario. \n\nOriginal Answer: Based on the additional context from PART II, CHAPTER I, SECTION 228, if you were driving under the influence of alcohol, you not only put your own life but also others at an immense risk. In his context, you may be punished for negligence and rash conduct that potentially endangered or caused harm to others. Specifically, for driving while drunk, you might face imprisonment and/or a fine. The extent of punishmen

In [22]:
[document.metadata['source']+':'+document.page_content for document in documents]

['Book II, PART III, CHAPTER I, SECTION 290:1. The punishment provided by section 289 (1) shall be doubled for an offence committed by the driver of any vehicle\n- a) who is drunk or drugged when driving ; or\n- b) who has not the license required for driving the vehicle in question ; or\n- c) who with intent to avoid his liability departs before being identified,\n1. Where such harm as is described in section 281 is caused unintentionally in the circumstances of the last foregoing subsection, the driver shall be punished with imprisonment for from 6 (six) months to 4 (four) years and with fine of from CFAF 10 000 (ten thousand) to CFAF 100 000 (one hundred thousand).\n2. For any offence against this section, the Court may disqualify the offender for up to five years from holding a driving license; and on subsequent conviction within the meaning of Section 88 of this Code may disqualify him for life or for any lesser term.\n3. In any case not covered by subsection (1) (c) of this secti